In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
##necessary imports
import json
import uuid
import os

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

In [3]:
##basically reads the dataframe and substitutes null values
class Reader(object):
    
    def __init__(self,file_name):
        self.file_name=file_name
        
    def run(self):
        df=pd.read_csv(self.file_name)
        df=df.fillna('') ##because elasticsearch rejects null values
        return df

In [4]:
##this class converts objects into vectors
class Tokenizer(object):
    def __init__(self):
        ##'all-MiniLM-L6-v2' is a pre-trained SentenceTransformer model for embedding text
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents] ##text is feeded into the pre-trained transformer as a list of strings
        sentence_embeddings = self.model.encode(sentences)
        _ = list(sentence_embeddings.flatten())##flatenning the dense vector returned by the encoder
        encod_np_array = np.array(_)
        encod_list = encod_np_array.tolist()
        return encod_list

##create index json for elasticsearch

PUT /posting
{
  "settings": {
    "index": {
      "number_of_shards": 20,
      "number_of_replicas": 1,
      "knn": {
        "algo_param": {
          "ef_search":40,
          "ef_construction":40,
          "m":"4"
        }
      }
    },
    "knn":true
  },
  "mappings": {
    "properties": {
      "vectors": {
        "type": "knn_vector",
        "dimension": 384
      }
    }
  }
}

In [16]:
##this essentially uploads data onto elasticsearch for comparison
class ElasticSearchImports(object):
    def __init__(self, df, index_name='posting'):
        self.df = df
        self.index_name = index_name
        self.es = Elasticsearch(timeout=600,hosts='https://localhost:8888')

    def run(self):

        elk_data = self.df.to_dict("records")

        for job in elk_data:
            try:self.es.index(index=self.index_name,body=job)
            except Exception as e:pass

        return True

In [14]:
##loading the articles dataset using the helper function
helper = Reader(file_name="articles_vectorized.csv")
df = helper.run()

In [21]:
##creating a "vectors" column in the dataset
tqdm.pandas()
helper_token = Tokenizer()
df["vectors"] = df["detail_desc"].progress_apply(helper_token.get_token)

100%|██████████████████████████████████████████████████████████████████████████| 105542/105542 [33:40<00:00, 52.23it/s]


In [18]:
helper_elk = ElasticSearchImports(df=df)
helper_elk.run()

C:\Users\abhsingh73\AppData\Local\Temp\ipykernel_20668\1607492833.py:6: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  self.es = Elasticsearch(timeout=600,hosts='https://localhost:9200')
C:\Users\abhsingh73\AppData\Local\Temp\ipykernel_20668\1607492833.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  try:self.es.index(index=self.index_name,body=job)


KeyboardInterrupt: 

In [28]:
df.to_csv('articles_vectorized.csv',index_label=False,index=False)

In [12]:
import socket
print(socket.gethostname())

WKWZTcDoYxEsaSi
